<a href="https://colab.research.google.com/github/suryawahyus/MachineLearning/blob/main/TaskWeek13/Multi_Layer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
transform = transforms.Compose([
    transforms.Resize((25, 25)),  # Mengubah ukuran gambar menjadi 25x25
    transforms.ToTensor(),        # Mengubah gambar menjadi tensor PyTorch
    transforms.Normalize((0.5,), (0.5,))  # Normalisasi gambar
])

# Muat dataset pelatihan dan pengujian
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Loader data
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 151194300.88it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 29530885.87it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 154922412.06it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 10560160.07it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(25*25, 300)  # Lapisan input ke hidden layer
        self.fc2 = nn.Linear(300, 10)     # Lapisan hidden ke output

    def forward(self, x):
        x = x.view(-1, 25*25)  # Flattening the images
        x = torch.relu(self.fc1(x))  # Aktivasi ReLU pada lapisan pertama
        x = self.fc2(x)
        return x

model = MLP()


In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss(reduction='sum')  # Inisialisasi di luar loop
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # Gunakan criterion
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)')



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 11):  # 10 epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Test set: Average loss: 0.1107, Accuracy: 9663/10000 (97%)
Test set: Average loss: 0.0889, Accuracy: 9722/10000 (97%)
Test set: Average loss: 0.0884, Accuracy: 9734/10000 (97%)
Test set: Average loss: 0.0835, Accuracy: 9746/10000 (97%)
Test set: Average loss: 0.0925, Accuracy: 9733/10000 (97%)
Test set: Average loss: 0.0857, Accuracy: 9749/10000 (97%)
Test set: Average loss: 0.0820, Accuracy: 9770/10000 (98%)
Test set: Average loss: 0.0818, Accuracy: 9769/10000 (98%)
Test set: Average loss: 0.0832, Accuracy: 9759/10000 (98%)
Test set: Average loss: 0.0790, Accuracy: 9782/10000 (98%)
